<a href="https://colab.research.google.com/github/EdBerg21/AI-Based-Fraud-Detection/blob/main/nvidia_fine_tuning_qwen_3_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setting Up

In [ ]:
%%capture
%pip install -U transformers
%pip install -U datasets
%pip install -U accelerate
%pip install -U peft
%pip install -U trl
%pip install -U bitsandbytes
%pip install huggingface_hub[hf_xet]

In [ ]:
from huggingface_hub import login
import os

hf_token = os.environ.get("HF_TOKEN")
login(hf_token)

## Loading the model and tokenizer

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
import torch

if torch.cuda.is_available():
    print("CUDA is available. GPU details:")
    print(torch.cuda.get_device_name(0))
else:
    print("CUDA is not available. bitsandbytes with CUDA backend will not work.")
    # You might need to check your GPU hardware, CUDA installation, and drivers.

CUDA is available. GPU details:
Tesla T4


In [ ]:
# Load tokenizer & model

model_dir = "nvidia/Nemotron-Research-Reasoning-Qwen-1.5B"

# Explicitly set the bitsandbytes backend to cuda
import os
os.environ['BITSANDBYTES_NOWELCOME'] = '1' # Suppress the welcome message
os.environ['CUDA_VISIBLE_DEVICES'] = '0' # Assuming you want to use the first GPU
# You might need to explicitly set the backend if the above doesn't work
#os.environ['BITSANDBYTES_BACKEND'] = 'cuda'

tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    quantization_config=bnb_config,
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer_config.json:   0%|          | 0.00/6.76k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/485 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Load tokenizer & model

model_dir = "nvidia/Nemotron-Research-Reasoning-Qwen-1.5B"

tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
!nvidia-smi

Thu Jun  5 15:50:48 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P0             27W /   70W |    3600MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Loading and processing the dataset

In [ ]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    inputs = examples["Question"]
    complex_cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for question, cot, response in zip(inputs, complex_cots, outputs):
        # Append the EOS token to the response if it's not already there
        if not response.endswith(tokenizer.eos_token):
            response += tokenizer.eos_token
        text = train_prompt_style.format(question, cot, response)
        texts.append(text)
    return {"text": texts}

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "FreedomIntelligence/medical-o1-reasoning-SFT",
    "en",
    split="train[0:2000]",
    trust_remote_code=True,
)
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)
dataset["text"][10]

README.md:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

medical_o1_sft.json:   0%|          | 0.00/58.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19704 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

"Below is an instruction that describes a task, paired with an input that provides further context. \nWrite a response that appropriately completes the request. \nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. \nPlease answer the following medical question. \n\n### Question:\nIn a patient with dermatomyositis as indicated by fatigue, muscle weakness, a scaly rash, elevated creatine kinase-MB, anti-Jo-1 antibodies, and perimysial inflammation, which type of cancer is most often associated with this condition?\n\n### Response:\n<think>\nAlright, so when I'm thinking about dermatomyositis, I know it's an inflammatory condition with muscle weakness and a telltale skin rash. It's sometimes linked to certain cancers. \n\nNow, I remember reading somewhere that when you have

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

## Model inference before fine-tuning

In [ ]:
inference_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>{}"""

In [ ]:
question = dataset[10]['Question']
inputs = tokenizer(
    [inference_prompt_style.format(question, "") + tokenizer.eos_token],
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    eos_token_id=tokenizer.eos_token_id,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(response[0].split("### Response:")[1])


<think>
Alright, let's see. The user provided a medical question about a patient with dermatomyositis and wants an answer based on the given context. 

First, I need to understand the question properly. The patient has dermatomyositis, which is characterized by fatigue, muscle weakness, a scaly rash, elevated creatine kinase-MB, anti-Jo-1 antibodies, and perimysial inflammation. The question is asking which type of cancer is most often associated with this condition.

Looking at the history provided earlier, the user provided a medical question and I responded with a step-by-step thought process. Now, the user is asking me to provide a similar detailed response but in a different format.

In the initial query, I had to write a response that completes the request based on the given context. Now, the user is asking me to respond with a detailed thought process or answer. So perhaps I need to outline the steps or provide a specific answer based on the given information.

But given the co

## Setting up the model

In [ ]:
from peft import LoraConfig, get_peft_model

# LoRA config
peft_config = LoraConfig(
    lora_alpha=16,                           # Scaling factor for LoRA
    lora_dropout=0.05,                       # Add slight dropout for regularization
    r=64,                                    # Rank of the LoRA update matrices
    bias="none",                             # No bias reparameterization
    task_type="CAUSAL_LM",                   # Task type: Causal Language Modeling
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],  # Target modules for LoRA
)

model = get_peft_model(model, peft_config)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments


# Training Arguments
training_arguments = TrainingArguments(
    output_dir="output",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    logging_steps=0.2,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="none"
)

# Initialize the Trainer
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset,
    peft_config=peft_config,
    data_collator=data_collator,
)

Converting train dataset to ChatML:   0%|          | 0/2000 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


## Model Training

In [ ]:
import gc, torch
gc.collect()
torch.cuda.empty_cache()
model.config.use_cache = False
trainer.train()

Step,Training Loss
200,1.866200
400,1.713400
600,1.700300
800,1.679900
1000,1.642900


TrainOutput(global_step=1000, training_loss=1.7205401611328126, metrics={'train_runtime': 4920.3694, 'train_samples_per_second': 0.406, 'train_steps_per_second': 0.203, 'total_flos': 1.3736585160889344e+16, 'train_loss': 1.7205401611328126})

## Model inference after fine-tuning

In [ ]:
question = dataset[10]['Question']
inputs = tokenizer(
    [inference_prompt_style.format(question, "") + tokenizer.eos_token],
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    eos_token_id=tokenizer.eos_token_id,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(response[0].split("### Response:")[1])


<think>
Okay, so we're looking at a patient with dermatomyositis. This is a condition where the body's muscles are weak and can't do their job properly. It's like the muscles are not working as they should, and it's often linked to some kind of inflammation. 

Now, let's think about what's going on with this patient. They're feeling really tired, and their muscles are not doing their job. That's a big clue. Plus, they have a scaly rash, which is pretty common in dermatitis. 

Oh, and there's this elevated creatine kinase-MB. That's a marker that tells us about inflammation. So, we're seeing a lot of inflammation going on here. 

And then there's anti-Jo-1 antibodies. These are a type of immune marker that can indicate a specific type of cancer. 

Now, let's think about what kind of cancer is most often associated with dermatomyositis. I know that there are a few types of skin cancers, but dermatomyositis is more about inflammation and muscle weakness. 

Oh, right! This is a classic si

In [ ]:
question = dataset[100]['Question']
inputs = tokenizer(
    [inference_prompt_style.format(question, "") + tokenizer.eos_token],
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    eos_token_id=tokenizer.eos_token_id,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(response[0].split("### Response:")[1])


<think>: A 25-year-old woman comes in to the ER with a rash that's pretty red and spread out. She's also feeling really sick, and she's been vomiting and having a fever for two days. That's a lot to take in. She's got a soaked tampon in her vagina, which is a big clue. 

Now, let's think about what could be causing this. The symptoms are pretty classic for something like toxic shock syndrome, which is a type of immune response. It's when the immune system is overwhelmed by a toxic substance, and it's usually a viral infection. 

So, what could be causing this? It could be a viral infection, like a virus that's causing the immune system to get really out of sync. But, wait, let's not jump to conclusions. We need to figure out what the toxin is. 

The fact that the rash is diffuse and erythematous tells us it's probably a viral infection. Now, what viruses are known to cause toxic shock syndrome? Let's think. There's a virus called T4, which is a viral infection that can cause this kind

## Saving the model

In [ ]:
new_model_name = "Qwen-3-32B-Medical-Reasoning"
model.push_to_hub(new_model_name)
tokenizer.push_to_hub(new_model_name)

Uploading...:   0%|          | 0.00/295M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Uploading...:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/EdBergJr1/Qwen-3-32B-Medical-Reasoning/commit/9b6b1f9b97ad86819c496e320b05fa5bdc110b55', commit_message='Upload tokenizer', commit_description='', oid='9b6b1f9b97ad86819c496e320b05fa5bdc110b55', pr_url=None, repo_url=RepoUrl('https://huggingface.co/EdBergJr1/Qwen-3-32B-Medical-Reasoning', endpoint='https://huggingface.co', repo_type='model', repo_id='EdBergJr1/Qwen-3-32B-Medical-Reasoning'), pr_revision=None, pr_num=None)